### часть 1
* реализация Gradient Descent для модели линейной регрессии

### часть 2
* реализация Back Propagation для MLP

In [ ]:
class MLPRegressor:
    def __init__( 
        self, 
        hidden layer_sizes=(100,), 
        learning_rate=0.001, 
        max_iter=10,
    ):
        pass
    def train(self, x, y): 
        pass
    def predict(self, X): 
        pass

### часть 1
* реализация Gradient Descent для модели линейной регрессии

In [ ]:
import numpy as np

class LinearRegressionGD:
    def __init__(self, learning_rate=0.01, max_iter=1000):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.theta = None

    def train(self, X, y):
        m, n = X.shape
        X = np.c_[np.ones((m, 1)), X]  # Добавляем столбец единиц для учета bias (свободного члена)
        self.theta = np.zeros(n + 1)

        for i in range(self.max_iter):
            gradients = (2/m) * X.T.dot(X.dot(self.theta) - y)
            self.theta -= self.learning_rate * gradients

    def predict(self, X):
        X = np.c_[np.ones((X.shape[0], 1)), X]  # Добавляем столбец единиц для учета bias
        return X.dot(self.theta)


### часть 2
* реализация Back Propagation для MLP

In [ ]:
import numpy as np

class MLPRegressor:
    def __init__(self, hidden_layer_sizes=(100,), learning_rate=0.001, max_iter=1000):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = []
        self.biases = []

    def _initialize_weights(self, input_size, output_size):
        layer_sizes = [input_size] + list(self.hidden_layer_sizes) + [output_size]
        self.weights = []
        self.biases = []
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i+1])))
    
    def _relu(self, Z):
        return np.maximum(0, Z)
    
    def _relu_derivative(self, Z):
        return Z > 0
    
    def _forward_propagation(self, X):
        activations = [X]
        pre_activations = []
        
        for i in range(len(self.weights)):
            Z = activations[-1].dot(self.weights[i]) + self.biases[i]
            pre_activations.append(Z)
            A = self._relu(Z) if i < len(self.weights) - 1 else Z  # Применение ReLU для скрытых слоев и линейной активации для выхода
            activations.append(A)
        
        return activations, pre_activations
    
    def _back_propagation(self, activations, pre_activations, y):
        m = y.shape[0]
        grads_W = [None] * len(self.weights)
        grads_b = [None] * len(self.biases)
        
        # Вычисление ошибки на выходе
        dZ = activations[-1] - y
        grads_W[-1] = activations[-2].T.dot(dZ) / m
        grads_b[-1] = np.sum(dZ, axis=0, keepdims=True) / m
        
        # Распространение ошибки назад по сети
        for i in reversed(range(len(grads_W) - 1)):
            dA = dZ.dot(self.weights[i+1].T)
            dZ = dA * self._relu_derivative(pre_activations[i])
            grads_W[i] = activations[i].T.dot(dZ) / m
            grads_b[i] = np.sum(dZ, axis=0, keepdims=True) / m
        
        return grads_W, grads_b
    
    def train(self, X, y):
        input_size = X.shape[1]
        output_size = y.shape[1] if len(y.shape) > 1 else 1
        self._initialize_weights(input_size, output_size)
        
        for i in range(self.max_iter):
            # Прямое распространение
            activations, pre_activations = self._forward_propagation(X)
            
            # Обратное распространение
            grads_W, grads_b = self._back_propagation(activations, pre_activations, y)
            
            # Обновление весов и смещений
            for j in range(len(self.weights)):
                self.weights[j] -= self.learning_rate * grads_W[j]
                self.biases[j] -= self.learning_rate * grads_b[j]
    
    def predict(self, X):
        activations, _ = self._forward_propagation(X)
        return activations[-1]